Saw that meme (see below) and could not resist mocking this a bit.

You will need a few additional libs to learn all that "rocket science" that they promised you will learn:
- `opencv`
- `fastai`

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
img = cv2.imread('meme.jpg')
img = cv2.resize(img, (640, 640))
# img = cv2.medianBlur(img, 3)
# img = cv2.GaussianBlur(img, (3, 3), 1)
img_H, img_W, channels = img.shape
img.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img, interpolation='nearest')
plt.title('Meme')
plt.show()

In [ ]:
windows = []
plt.figure(1, figsize=(10, 10))
tmp = img # for drawing a rectangle
step = 32
(window_W, window_H) = (32, 32) # window size
for x in range(0, img_W - window_W + step, step):
    for y in range(0, img_H - window_H + step, step):
        window = img[x:x + window_W, y:y + window_H, :]
        windows.append(window)
        cv2.rectangle(tmp, (x, y), (x + window_W, y + window_H), (255, 0, 0), 2) # draw rectangle on image
        plt.imshow(np.array(tmp).astype('uint8'))
plt.show()

In [ ]:
def get_img_features(img):
    r, g, b = cv2.split(img)
    r_stats = np.array(pd.Series(r.flatten()).describe())
    g_stats = np.array(pd.Series(g.flatten()).describe())
    b_stats = np.array(pd.Series(b.flatten()).describe())
    gray = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
    blurred1 = cv2.GaussianBlur(gray, (3, 3), 1).flatten()
    blurred2 = cv2.GaussianBlur(gray, (7, 7), 1).flatten()
    flipped_h1 = cv2.flip(blurred1, 0).flatten()
    flipped_h2 = cv2.flip(blurred2, 0).flatten()
    flipped_v1 = cv2.flip(blurred1, 1).flatten()
    flipped_v2 = cv2.flip(blurred2, 1).flatten()
    return np.concatenate((r_stats, g_stats, b_stats, blurred1, blurred2, flipped_h1, flipped_h2, flipped_v1, flipped_v2))

In [ ]:
result = [get_img_features(window) for window in windows]

In [ ]:
window_df = pd.DataFrame(result)
window_df

In [ ]:
from sklearn import preprocessing
x = window_df.values
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_2d = pca.fit_transform(df)
X_2d

In [ ]:
data = pd.DataFrame(X_2d, columns=['fst', 'snd'])
plt.figure(figsize=(12,8))
plt.scatter(data['fst'], data['snd'], alpha=.5)
plt.title('Data in PCA space')
plt.xlabel('PC1')
plt.ylabel('PC2')
top_plot = plt.gca()
plt.show()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
pca = PCA(n_components=10)
test = pca.fit_transform(df)

max_n = 10
plt.figure(figsize=(10, 8))
wcss = []
for i in range(1, max_n):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(test)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, max_n), wcss)
plt.title('The Elbow Method (Original)')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
nc = 3
kmeans = KMeans(n_clusters=nc, random_state=42)  
kmeans.fit(test)

colours = matplotlib.rcParams['axes.prop_cycle'].by_key()['color']
cluster_ids = list(np.arange(nc))
cluster_names = [f'Cluster {cluster_id}' for cluster_id in cluster_ids]
labels = kmeans.labels_
centers = kmeans.cluster_centers_

plt.figure(figsize=(12,8))
for color, c_id, c_name in zip(colours, cluster_ids, cluster_names):
    plt.scatter(X_2d[labels == c_id, 0], X_2d[labels == c_id, 1], color=color, alpha=.3, lw=1, label=c_name)
    
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)
plt.axis('equal')
plt.legend(loc='center left', scatterpoints=3, bbox_to_anchor=[1.01, 0.5])
plt.title('Labelled data in PCA space')
plt.xlabel('PC1')
plt.ylabel('PC2')
top_plot = plt.gca()
plt.show()

In [ ]:
labels

In [ ]:
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

plt.figure(1, figsize=(10, 10))
tmp = np.zeros(img.shape, np.uint8) # for drawing a rectangle
step = 32
(window_W, window_H) = (32, 32) # window size
counter = 0
for x in range(0, img_W - window_W + step, step):
    for y in range(0, img_H - window_H + step, step):
        
        window = img[x:x + window_W, y:y + window_H, :]
        
        label = labels[counter]
        
        # Blue rectangle
        if label == 0:           
            cv2.rectangle(tmp, (x, y), (x + window_W, y + window_H), BLUE, -1) 
            
        # Blue rectangle
        if label == 1:           
            cv2.rectangle(tmp, (x, y), (x + window_W, y + window_H), GREEN, -1)
        
        # RED rectangle
        if label == 2:           
            cv2.rectangle(tmp, (x, y), (x + window_W, y + window_H), RED, -1)
            
        counter += 1
            
out = cv2.addWeighted(img, 0.75, tmp, 0.25, 1)
plt.imshow(out)
plt.show()

Cleaner option:

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

img = cv2.imread('meme2.png')
img = cv2.resize(img, (640, 640))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

r, g, b = cv2.split(img)
r = r.flatten()
g = g.flatten()
b = b.flatten()
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(r, g, b)
plt.show()

In [ ]:
vectorized = img.reshape((-1,3))
vectorized = np.float32(vectorized)

In [ ]:
vectorized.shape

In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

In [ ]:
K = 4
attempts = 10
ret, label, center = cv2.kmeans(vectorized, K, None,criteria,attempts,cv2.KMEANS_PP_CENTERS)

In [ ]:
center = np.uint8(center)

In [ ]:
res = center[label.flatten()]
result_image = res.reshape((img.shape))

In [ ]:
result_image.shape

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title('Original Image')
plt.xticks([])
plt.yticks([])
plt.subplot(1,2,2)
plt.imshow(result_image)
plt.title('Segmented Image when K = %i' % K)
plt.xticks([])
plt.yticks([])
plt.show()